In [1]:
#importing libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first='true')

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
independent=dataset[['Age','EstimatedSalary','Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [7]:
dependent=dataset['Purchased']
dependent

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [8]:
#split train and test set
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

C:\Users\sindhiya maria\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
     'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
     'penalty':['l1', 'l2', 'elasticnet'],
     'class_weight':[ 'balanced']
     } 




# Create the GridSearchCV object
grid = GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

# Fit the model
grid.fit(x_train, y_train)


Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\sindhiya maria\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan 0.81401351        nan        nan        nan 0.81025909
 0.81365168 0.81365168 0.81365168        nan 0.81365168 0.81365168
        nan        nan        nan        nan        nan        nan]
  warnings.warn(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [11]:
# print best parameter after tuning
re=grid.cv_results_
grid_predictions = grid.predict(x_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,grid_predictions)

# print classification report 
from sklearn.metrics  import classification_report
clf_report = classification_report(y_test, grid_predictions)


In [12]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}: 0.8818571308560388


In [13]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[74 11]
 [ 5 44]]


In [14]:
print("The Report:\n",clf_report)

The Report:
               precision    recall  f1-score   support

           0       0.94      0.87      0.90        85
           1       0.80      0.90      0.85        49

    accuracy                           0.88       134
   macro avg       0.87      0.88      0.87       134
weighted avg       0.89      0.88      0.88       134



In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9493397358943576

In [16]:
table =pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000,0.000000,0.000000,balanced,l1,lbfgs,"{'class_weight': 'balanced', 'penalty': 'l1', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,12
1,0.000502,0.001003,0.003124,0.006248,balanced,l1,liblinear,"{'class_weight': 'balanced', 'penalty': 'l1', ...",0.777778,0.815037,0.756031,0.815056,0.906166,0.814014,0.051321,1
2,0.000000,0.000000,0.000000,0.000000,balanced,l1,newton-cg,"{'class_weight': 'balanced', 'penalty': 'l1', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,16
3,0.006255,0.007660,0.000000,0.000000,balanced,l1,newton-cholesky,"{'class_weight': 'balanced', 'penalty': 'l1', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,15
4,0.000000,0.000000,0.000000,0.000000,balanced,l1,sag,"{'class_weight': 'balanced', 'penalty': 'l1', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
5,0.015660,0.000000,0.000000,0.000000,balanced,l1,saga,"{'class_weight': 'balanced', 'penalty': 'l1', ...",0.777778,0.815037,0.756031,0.796284,0.906166,0.810259,0.051792,7
6,0.009581,0.007825,0.010375,0.007127,balanced,l2,lbfgs,"{'class_weight': 'balanced', 'penalty': 'l2', ...",0.757701,0.815037,0.756031,0.833323,0.906166,0.813652,0.055495,2
7,0.011077,0.007799,0.001403,0.001718,balanced,l2,liblinear,"{'class_weight': 'balanced', 'penalty': 'l2', ...",0.757701,0.815037,0.756031,0.833323,0.906166,0.813652,0.055495,2
8,0.000000,0.000000,0.000000,0.000000,balanced,l2,newton-cg,"{'class_weight': 'balanced', 'penalty': 'l2', ...",0.757701,0.815037,0.756031,0.833323,0.906166,0.813652,0.055495,2
9,0.000000,0.000000,0.000000,0.000000,balanced,l2,newton-cholesky,"{'class_weight': 'balanced', 'penalty': 'l2', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,17


In [18]:
age_input=float(input("Age:"))

Gender_male_input=int(input("Gender_Male 0 or 1:"))

EstimatedSalary_input=int(input("EstimatedSalary:"))


Age:35
Gender_Male 0 or 1:1
EstimatedSalary:20000


In [19]:

Future_Prediction=grid.predict([[age_input,Gender_male_input,EstimatedSalary_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
